# ecdc.europa.eu
https://www.ecdc.europa.eu/en/covid-19/data

* [Opendata europe -nationalcasedeath json](https://opendata.ecdc.europa.eu/covid19/nationalcasedeath/json/)


# Reference Data
[Variable_Dictionary_VaccineTracker](https://www.ecdc.europa.eu/sites/default/files/documents/Variable_Dictionary_VaccineTracker-20-08-2021.pdf)

In [ ]:
case class CovidVaccine(code:String, name:String)
val covidVaccineDF= spark.createDataFrame(
    Seq(CovidVax("AZ","Vaxzevria – AstraZeneca")
,CovidVax("BBIBP","Sinopharm")
    
,CovidVax("BECNBG","Inactivated – Beijing CNBG")
,CovidVax("BHACOV","Covaxin – Bharat")
,CovidVax("COM","Comirnaty – Pfizer/BioNTech")
,CovidVax("JANSS","Ad26.COV 2.5 – Janssen")
,CovidVax("HAYATVAC","Hayat VAC")
,CovidVax("MOD","mRNA-1273 – Moderna")
,CovidVax("QAZVAQ","QazCovid-In")
,CovidVax("SIICOV","Covishield – SII") 
,CovidVax("SIN","CoronaVac – Sinovac")
,CovidVax("SPU","Sputnik V - Gamaleya")
,CovidVax("SRCVB","EpiVacCorona – SRCVB")
,CovidVax("WUCNBG","Inactivated – WuhanCNBG")
,CovidVax("UNK","UNKNOWN")
,CovidVax("ZFUZ","Sino-Uzbek - ZF-UZ-VAC")))

covidVaccineDF.show

# Vaccine tracker
[Opendata europe vaccine_tracker &#10515;](https://opendata.ecdc.europa.eu/covid19/vaccine_tracker/json)


In [ ]:
case class VaccineTracker (
    rowNumber:BigInt
    , YearWeekISO: String
    , firstDose: Integer
    , firstDoseRefused: Integer
    , secondDose: Integer
    , unknownDose: Integer
    , numberDosesReceived: Integer
    , region: String
    , population: Integer
    , reportingCountry: String
    , targetGroup: String
    , vaccine : String
    , denominator: Integer)

In [ ]:
import org.apache.spark.sql._
import org.apache.spark.sql.functions._
import org.apache.spark.sql.types._

val vaccineTrackerDF = spark.read.option("header", "true")
    .csv("file:///home/taccart/Downloads/ECDC/vaccing_tracker.csv") 
    .withColumn("firstDose", col("FirstDose").cast(IntegerType))
    .withColumn("firstDoseRefused", col("FirstDoseRefused").cast(IntegerType))
    .withColumn("secondDose", col("SecondDose").cast(IntegerType))
    .withColumn("unknownDose", col("UnknownDose").cast(IntegerType))
    .withColumn("numberDosesReceived", col("NumberDosesReceived").cast(IntegerType))
    .withColumn("population", col("Population").cast(IntegerType))
    .withColumn("denominator", col("Denominator").cast(IntegerType))
    .withColumn("rowNumber", monotonically_increasing_id())
    .na.fill(0)
.as[VaccineTracker]

vaccineTrackerDF.createOrReplaceTempView("TVaxTracker")



In [ ]:
spark.sql("select vaccine, count(*),min (YearWeekISO), max(YearWeekISO) from tvaxtracker group by vaccine").show

In [ ]:
spark.sql("select distinct VAX_MANU from tvax where vax_type= 'COVID19'").show